## Table of contents:
* Initial Setup
* Data Loading and Preprocessing
    * Data Variables
* Model Instantiation
* Optimizer and Scheduler
* Train and Evaluation Loop
    * Save the model and plot the losses (**1 Point**)
* Test Loop  
* Human Pose Visualization (**2 Points**)
* Report and Parameter Fine-Tuning Analysis  (**4 Points**)
* Calculating MPJPE for a Specific Frame  (**2 Points**)
* Iterative Mechanism (**3 Points**)
* YOUR custom model (**3 Points**)
    * Performance BONUS (**Up to 2 Points**)



## Initial Setup
Run the following two cellls to sync with Google Drive only if you run from Google Colab.

*Note: we recommend using Google Colab for this specific homework, since the training phase will require a GPU*

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)

In [ ]:
# # Insert your path here
# %cd "/content/drive/MyDrive/Colab Notebooks/Homework 3/Practice"

Welcome to this guide on training, testing, and fine-tuning a deep learning model. Deep learning is at the forefront of artificial intelligence, with applications spanning image recognition, natural language processing, and more.

Throughout this assignment, you'll:

1. **Prepare Data:** Preprocess and load the data.

2. **Use Neural Networks:** Instantiate a neural network architecture.

3. **Train Models:** Utilize optimization, loss functions, and backpropagation.

4. **Evaluate Performance:** Assess model performance, prevent overfitting, and underfitting.

5. **Fine-Tune Models:** Explore hyperparameter tuning.

In [ ]:
from utils import h36motion3d as datasets
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import torch.optim as optim
import torch.autograd
import torch
import numpy as np
from utils.loss_funcs import *
from utils.data_utils import define_actions
from utils.h36_3d_viz import visualize
import time

import torch.nn.functional as F

#reproducibility stuff
torch.manual_seed(0)
torch.cuda.manual_seed(0)
np.random.seed(0)

torch.backends.cudnn.deterministic=True

In [ ]:
# Use GPU if available, otherwise stick with cpu
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device: %s'%device)
#print('Using device:', device,  '- Type:', torch.cuda.get_device_name(0))

## Data Loading and Preprocessing

For this homework, you will use [Human3.6M](https://vision.imar.ro/human3.6m/pami-h36m.pdf), which is a large-scale dataset of 3.6 million accurate 3D human poses acquired by recording the performance of five female and six male subjects under four different viewpoints. The dataset includes:
- Synchronized image.
- Human motion capture.
- Time of flight (depth) data.
- Accurate 3D body scans of all the subject actors involved.

The dataset aims to provide diverse motions and poses encountered in typical human activities, with additional data to train realistic human sensing systems.

![picture](https://drive.google.com/uc?export=view&id=1nyD1_F3r1ctexKFGDmhy2Q9SX_2Z_bpS)

For this assignment, we will leverage the rich **motion data** (See in the figure above) provided by H3.6M to perform a task known as *motion prediction*. Motion prediction involves using historical motion data to forecast future movements. This task is fundamental in human-robot interaction, animation, and sports analytics applications.

### Data Variables

Each created sequence has the shape (35, 17, 3), where:
- Number of observed sequences and number of sequences to predict: $N_{obs}+N_{pred} = 10 + 25 = 35$;
- Number of body joints to consider: $J=22$;
- Spatial coordinates: $(x,y,z) = 3$.


The original data provides high-resolution progressive scan videos at 50 Hz. However, the dataset has been downsampled to 25 Hz for research purposes. This means that 25 frames of motion data are provided per second.

*Note: the figure above shows 18 joints, however the dataset contains 32. For this specific case we will consider 22 joints, ignoring some of the finer ones (e.g. foot tip, hand tip, etc)*


In [ ]:
# # Arguments to setup the datasets
datas = 'h36m' # dataset name
data_path = './data/h3.6m/h3.6m/dataset'
input_n=10 # number of frames to train on (default=10)
output_n=25 # number of frames to predict on
input_dim=3 # dimensions of the input coordinates(default=3)
skip_rate=1 # # skip rate of frames
joints_to_consider=22


#FLAGS FOR THE TRAINING
mode='train' #choose either train or test mode

batch_size_test=8
model_path= './checkpoints/' # path to the model checkpoint file

actions_to_consider_test='all' # actions to test on.
model_name = datas+'_3d_'+str(output_n)+'frames_ckpt' #the model name to save/load

#FLAGS FOR THE VISUALIZATION
actions_to_consider_viz='all' # actions to visualize
visualize_from='test'
n_viz=2

Load Dataset

*Note: It will take you ~ 5 minutes*

In [ ]:
# Load Data
print('Loading Train Dataset...')
dataset = datasets.Datasets(data_path,input_n,output_n,skip_rate, split=0)
print('Loading Validation Dataset...')
vald_dataset = datasets.Datasets(data_path,input_n,output_n,skip_rate, split=1)

#! Note: Ignore warning:  "VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences"

Following we create a torch dataloader that create the batches for each epoch.

In [ ]:
batch_size=256

print('>>> Training dataset length: {:d}'.format(dataset.__len__()))
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)#

print('>>> Validation dataset length: {:d}'.format(vald_dataset.__len__()))
vald_loader = DataLoader(vald_dataset, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)

## Model instantiation


Each sequence comprises an **observed** part to train the Encoder and a part that attempts to predict the **future** sequence, the Decoder.

Generally, the standard setup plans to use the first 10 sequences of poses ($N_{obs}=10$) for the observation and the following 25 ($N_{pred} = 25$) for the prediction.


We create an instance of a custom Spatio-Temporal transformer with the chosen configuration.

(*Note: explore the model in ./models/sttr/sttformer.py*)

Then we allocate it to the GPU for forward and backward accelerated computation.

In [ ]:
from models.sttr.sttformer import Model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device: %s'%device)

n_heads = 1

model = Model(num_joints=joints_to_consider,
                 num_frames=input_n, num_frames_out=output_n, num_heads=n_heads,
                 num_channels=3, kernel_size=[3,3], use_pes=True).to(device)

print('total number of parameters of the network is: '+str(sum(p.numel() for p in model.parameters() if p.requires_grad)))

## Optimizer and Scheduler

As we embark on training deep learning models for motion prediction using the H3.6M dataset, it's essential to recognize several key parameters and components that significantly impact the training phase:

- **Learning Rate:** This parameter determines the convergence speed during optimization.

- **Batch Size:** It influences model generalization and training efficiency.

- **Number of Epochs:** The number of training iterations affects model learning.

- **Loss Function:** The choice of loss function directly affects learning and final performance.

- **Optimizer:** The optimization algorithm used (e.g., Adam, SGD) impacts gradient descent during training.

- **Milestones and Gamma:** These parameters control learning rate schedules, allowing for adaptive adjustments during training.

- **Weight Decay:** It regulates the impact of model parameters during optimization.

- **Scheduler:** Scheduler strategies (e.g., StepLR, ReduceLROnPlateau) manage learning rate adaptation during training.

In [ ]:
# Arguments to setup the optimizer
lr=1e-01 # learning rate
use_scheduler=True # use MultiStepLR scheduler
milestones=[10,30]   # the epochs after which the learning rate is adjusted by gamma
gamma=0.1 #gamma correction to the learning rate, after reaching the milestone epochs
weight_decay=1e-05 # weight decay (L2 penalty)
optimizer=optim.Adam(model.parameters(),lr=lr,weight_decay=weight_decay)

if use_scheduler:
    scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=milestones, gamma=gamma)

In [ ]:
clip_grad=None # select max norm to clip gradients
# Argument for training
n_epochs=41
log_step = 200

## Train and Evaluation Loop

The **loss** and **metric** used during training and evaluation respectively, compare the predicted joint positions to ground truth joint positions for all frames, which is typically referred to as the **Average Mean Per
Joint Position Error (A-MPJPE)** can be seen as an $L_2$. This loss quantifies the dissimilarity between the predicted and ground truth joint positions by measuring the squared Euclidean distance between corresponding joint positions.

\begin{align*}
A-MPJPE &= \frac{1}{N_{pred}} \sum_{i=1}^{N_{pred}} \left(\frac{1}{J} \sum_{j=1}^{J} \left\| P_{\text{predicted}_{t,j}} - P_{\text{gt}_{t,j}} \right\|^2\right)
\end{align*}

$$where:$$

\begin{align*}
P_{\text{predicted}} &: \text{Set of predicted joint positions estimated by the model.} \\
P_{\text{gt}} &: \text{Corresponding set of ground truth joint positions.} \\
\end{align*}

*Note: If you restart the training for any reason, remember to instantiate the model and the optimizer again. This will avoid continuing the training with the initialized weights of the previous one*


### Save the model and plot the losses (1 Point)

**Objective:** In this exercise, you will practice implementing a code snippet to save your deep learning model's checkpoints and visualize the training and validation loss on the same plot every 5 epochs during model training.

Your task is to implement the following:

- Set up a mechanism to save the model's checkpoints (weights and architecture) during training. These checkpoints should be saved periodically, say, **every 5 epochs**.

- Create a plot displaying the training and validation losses on the same graph. The x-axis should represent the number of epochs, and the y-axis should represent the loss values. The training and validation losses should be plotted as separate lines on the same graph.

- Ensure that the code saves the model's checkpoints in a specified directory, including the model's architecture and weights, and that the loss plot is displayed.

Analyze the loss plot to gain insights into how your model is learning over time and whether there are any signs of overfitting or underfitting.

*Note: see the Pytorch Documentation on how to save your model's checkpoints.*


In [ ]:
import os
from os import path
import copy

def train(data_loader,vald_loader, path_to_save_model=None):
  # Creation of a folder where to save plots
  if path.exists(path_to_save_model + "plots/") == False:
    os.mkdir(path_to_save_model + "plots/")

  train_loss = []
  val_loss = []
  val_loss_best = 1000

  dim_used = np.array([6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 21, 22, 23, 24, 25,
                    26, 27, 28, 29, 30, 31, 32, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45,
                    46, 47, 51, 52, 53, 54, 55, 56, 57, 58, 59, 63, 64, 65, 66, 67, 68,
                    75, 76, 77, 78, 79, 80, 81, 82, 83, 87, 88, 89, 90, 91, 92])

  for epoch in range(n_epochs-1):
      running_loss=0
      n=0
      model.train()
      for cnt,batch in enumerate(data_loader):
          batch=batch.float().to(device)
          batch_dim=batch.shape[0]
          n+=batch_dim

          sequences_train=batch[:, 0:input_n, dim_used].view(-1,input_n,len(dim_used)//3,3).permute(0,3,1,2)
          sequences_gt=batch[:, input_n:input_n+output_n, dim_used].view(-1,output_n,len(dim_used)//3,3)

          optimizer.zero_grad()
          sequences_predict=model(sequences_train).view(-1, output_n, joints_to_consider, 3)


          loss=mpjpe_error(sequences_predict,sequences_gt)


          if cnt % log_step == 0:
            print('[Epoch: %d, Iteration: %5d]  training loss: %.3f' %(epoch + 1, cnt + 1, loss.item()))

          loss.backward()
          if clip_grad is not None:
            torch.nn.utils.clip_grad_norm_(model.parameters(),clip_grad)

          optimizer.step()
          running_loss += loss*batch_dim

      train_loss.append(running_loss.detach().cpu()/n)
      model.eval()
      with torch.no_grad():
          running_loss=0
          n=0
          for cnt,batch in enumerate(vald_loader):
              batch=batch.float().to(device)
              batch_dim=batch.shape[0]
              n+=batch_dim


              sequences_train=batch[:, 0:input_n, dim_used].view(-1,input_n,len(dim_used)//3,3).permute(0,3,1,2)
              sequences_gt=batch[:, input_n:input_n+output_n, dim_used].view(-1,output_n,len(dim_used)//3,3)

              sequences_predict=model(sequences_train).view(-1, output_n, joints_to_consider, 3)
              loss=mpjpe_error(sequences_predict,sequences_gt)

              if cnt % log_step == 0:
                        print('[Epoch: %d, Iteration: %5d]  validation loss: %.3f' %(epoch + 1, cnt + 1, loss.item()))
              running_loss+=loss*batch_dim
          val_loss.append(running_loss.detach().cpu()/n)
          if running_loss/n < val_loss_best:
            val_loss_best = running_loss/n
            best_model = copy.deepcopy(model)

      if use_scheduler:
        scheduler.step()

      # save and plot model every 5 epochs
      '''
      Insert your code below. Use the argument path_to_save_model to save the model to the path specified.
      '''
      if save_and_plot and (epoch+1)%5==0 and epoch!=0: # Save the model and display the losses every 5 epochs
        torch.save(model.state_dict(), f'{path_to_save_model}/h36m_3d_25frames_ckpt_epoch_{epoch+1}_q1.pt')
        # we know that in this way we save only the parameters, but we tried using torch.jit.scripts and it didn't work
        # in particular, it gives us this error "RuntimeError: Can't redefine method: forward on class: __torch__.torch.nn.modules.dropout.Dropout (of Python compilation unit at: 0x57b5c8a24d70)"
        # we search it on google but we didn't find any solution on any forum, so we stuck to the standard torch.save()

        # Plot the training and validation loss
        fig, ax = plt.subplots()
        x_tick_freq = 1 if epoch < 16 else 2 if epoch<26 else 4
        epochs = range(1, epoch+2, x_tick_freq) # The epoch numbering in the plots starts from 1 in order to stick with the code above
        x_list = list(range(1, len(train_loss)+1))
        ax.plot(x_list, train_loss, 'r', label='Train loss') # Line that displays the train loss
        ax.plot(x_list, val_loss, 'g', label='Val loss') # Line that displays the validation loss
        params = "Hyperparams: lr-> " + str(lr) + ", ml->" + str(milestones) + ", wd->" + str(weight_decay)
        ax.set_xticks(list(epochs))
        ax.set_xlabel('Epochs \n ' + params)
        ax.set_ylabel('Loss')
        plt.legend()
        plt.title('Loss History', fontsize=12)

        plt.savefig(path_to_save_model + "plots/loss_epoch_"+str(epoch+1)+"-"+str(lr)+"-"+str(milestones)+"-"+str(weight_decay)+".png", bbox_inches='tight')
        plt.show()
  torch.save(best_model.state_dict(), f'{path_to_save_model}/h36m_3d_25frames_ckpt_best_model.pt')

In [ ]:
save_and_plot = True # save the model and plot the loss. Change to True if you want to save the model and plot the loss

# launch training
train(data_loader,vald_loader, path_to_save_model=model_path)

## Test Loop

After training and validation, the test loop is the final phase that evaluates the model's performance on an entirely independent dataset known as the test dataset. This dataset is distinct from the training and validation data, ensuring unbiased assessment. The test loop provides a reliable estimate of how well the model will perform in real-world scenarios, confirming that any improvements observed during training and validation are not due to overfitting or chance. It's a crucial step before deploying the model in practical applications.

In [ ]:
gt_preds = [] # Array to store tuples (ground-truth, prediction)

In [ ]:
def test(ckpt_path=None):
    model.load_state_dict(torch.load(ckpt_path, map_location=torch.device('cpu')))
    print('model loaded')
    model.eval()
    accum_loss=0
    n_batches=0 # number of batches for all the sequences
    actions=define_actions(actions_to_consider_test)
    dim_used = np.array([6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 21, 22, 23, 24, 25,
                      26, 27, 28, 29, 30, 31, 32, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45,
                      46, 47, 51, 52, 53, 54, 55, 56, 57, 58, 59, 63, 64, 65, 66, 67, 68,
                      75, 76, 77, 78, 79, 80, 81, 82, 83, 87, 88, 89, 90, 91, 92])
    # joints at same loc
    joint_to_ignore = np.array([16, 20, 23, 24, 28, 31])
    index_to_ignore = np.concatenate((joint_to_ignore * 3, joint_to_ignore * 3 + 1, joint_to_ignore * 3 + 2))
    joint_equal = np.array([13, 19, 22, 13, 27, 30])
    index_to_equal = np.concatenate((joint_equal * 3, joint_equal * 3 + 1, joint_equal * 3 + 2))
    totalll=0
    counter=0
    for action in actions:
      running_loss=0
      n=0
      dataset_test = datasets.Datasets(data_path,input_n,output_n,skip_rate, split=2,actions=[action])
      #print('>>> test action for sequences: {:d}'.format(dataset_test.__len__()))

      test_loader = DataLoader(dataset_test, batch_size=batch_size_test, shuffle=False, num_workers=0, pin_memory=True)
      for cnt,batch in enumerate(test_loader):
        with torch.no_grad():

          batch=batch.to(device)
          batch_dim=batch.shape[0]
          n+=batch_dim


          all_joints_seq=batch.clone()[:, input_n:input_n+output_n,:]

          sequences_train=batch[:, 0:input_n, dim_used].view(-1,input_n,len(dim_used)//3,3).permute(0,3,1,2)
          sequences_gt=batch[:, input_n:input_n+output_n, :]


          running_time = time.time()
          sequences_predict=model(sequences_train).view(-1, output_n, joints_to_consider, 3)
          #sequences_predict = model(sequences_train)
          totalll += time.time()-running_time
          counter += 1
          sequences_predict=sequences_predict.contiguous().view(-1,output_n,len(dim_used))

          all_joints_seq[:,:,dim_used] = sequences_predict


          all_joints_seq[:,:,index_to_ignore] = all_joints_seq[:,:,index_to_equal]

          '''
          The code below has been added for the visualization task
          '''
          # Since the exercise ask to visualize 22 joints (connections found in the file at "./models/skeleton_connection.py.")
          # and the preditions already consider them, we exploit "dim_used," "index_to_ignore," and "index_to_equal" to
          # equalize the "joint to ignore" ones in the ground-truth. This allows us to plot the ground-truth and the prediction
          # together and analyze the differences.

          gt_eq_joints = sequences_gt.clone()
          gt_eq_joints[:,:,index_to_ignore] = gt_eq_joints[:,:,index_to_equal] # Equalize some nodes of the gt to plot only 22 joints out of 32
          #gt_eq_joints = gt_eq_joints[:,:,dim_used]

          gt_pred = (gt_eq_joints.view(-1, output_n, 32, 3), all_joints_seq.view(-1, output_n, 32, 3))
          gt_preds.append(tuple(gt_pred)) #Add tuples (ground-truth, prediction) to the list gt_preds
          ''''''

          loss=mpjpe_error(all_joints_seq.view(-1,output_n,32,3),sequences_gt.view(-1,output_n,32,3))
          running_loss+=loss*batch_dim
          accum_loss+=loss*batch_dim

      #print('loss at test subject for action : '+str(action)+ ' is: '+ str(running_loss/n))
      print(str(action),': ', str(np.round((running_loss/n).item(),1)))
      n_batches+=n
    print('Average: '+str(np.round((accum_loss/n_batches).item(),1)))
    print('Prediction time: ', totalll/counter)

*Note: Your results should be better than 95 millimiters on average*

In [ ]:
ckpt_path = './checkpoints/h36m_3d_25frames_ckpt_best_model.pt' # load the best model
test(ckpt_path)

## Human Pose Visualization  (**2 Points**)

The qualitative results are as important as the quantitative ones. In this section, you will visualize and compare the predicted poses with the ground truth ones.
For simplicity, you will visualize only the first predicted pose and the ground truth.

Ideally, the same plot should show the predicted pose in red and the ground truth one in green.

*Note: you will find which nodes are connected in the file ./models/skeleton_connection.py*

In [ ]:
'''
Insert your code below
'''
def visualize_pose(gt, predict):
    # Convert tensor to numpy
    gt = gt.cpu().numpy()
    predict = predict.cpu().numpy()

    # Initialize the 3d plot
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    # Define connections to draw the skeleton (taken from the file ./models/skeleton_connection.py)
    connect = [
            (1, 2), (2, 3), (3, 4), (4, 5),
            (6, 7), (7, 8), (8, 9), (9, 10),
            (0, 1), (0, 6),
            (6, 17), (17, 18), (18, 19), (19, 20), (20, 21), (21, 22),
            (1, 25), (25, 26), (26, 27), (27, 28), (28, 29), (29, 30),
            (24, 25), (24, 17),
            (24, 14), (14, 15)
    ]

    # Plot all joints of the groud-truth (in green) and the prediction (in red)
    for i, c in enumerate(connect):
        joint1_gt = gt[c[0]]
        joint2_gt = gt[c[1]]
        joint1_pred = predict[c[0]]
        joint2_pred = predict[c[1]]
        ax.plot([joint1_gt[0], joint2_gt[0]], [joint1_gt[1], joint2_gt[1]], [joint1_gt[2], joint2_gt[2]], color='green', linewidth=2)
        ax.plot([joint1_pred[0], joint2_pred[0]], [joint1_pred[1], joint2_pred[1]], [joint1_pred[2], joint2_pred[2]], color='red', linewidth=2)

    # Set the plot limits
    xroot, yroot, zroot = gt[0]
    RADIUS = 700
    ax.set_xlim3d([-RADIUS + xroot, RADIUS + xroot])
    ax.set_zlim3d([-RADIUS + zroot, RADIUS + zroot])
    ax.set_ylim3d([-RADIUS + yroot, RADIUS + yroot])

    # Rotate the plot so that the human looks toward the camera
    ax.view_init(elev=90, azim=0, roll=90)

    # Remove the ticks axes
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_zticks([])

    # Set the pane color to white
    ax.xaxis.pane.fill = False
    ax.yaxis.pane.fill = False
    ax.zaxis.pane.fill = False

    # Set edge color to white
    ax.xaxis.pane.set_edgecolor('w')
    ax.yaxis.pane.set_edgecolor('w')
    ax.zaxis.pane.set_edgecolor('w')

    # Remove the grid
    ax.grid(False)
    plt.axis('off')

    gt_patch = mpatches.Patch(color='green', label='Ground-truth pose')
    pred_patch = mpatches.Patch(color='red', label='Prediction pose')
    plt.legend(handles=[gt_patch, pred_patch])

    # Show the plot
    plt.show()

In [ ]:
# Visualize (the first frame of) the first pose as requested by the text
first_pose = gt_preds[0]
print(first_pose[0][0][0].size())
visualize_pose(first_pose[0][0][0], first_pose[1][0][0])

## Report and Parameter Fine-Tuning Analysis  (**4 Points**)

**Objective:** In this exercise, you will analyze the results obtained from a deep learning model you previously trained and perform parameter fine-tuning to optimize its performance. The key considerations are learning rate, milestones, and weight decay. **You will also use tables and plots to visualize and interpret the outcomes.**

**Instructions:**

1. **Analysis:** Analyze the generated report and answer the following questions:
   - Is there evidence of overfitting or underfitting in the initial training results?
   - Are there fluctuations in training and validation loss or accuracy? If so, what might be causing them?
   - What can you infer from the initial learning rate, milestones, and weight decay settings?

2. **Parameter Fine-Tuning:** Based on your analysis, perform parameter fine-tuning to optimize model performance. Adjust the following parameters:
   - **Learning Rate:** Experiment with different learning rates (higher and lower values) to find an optimal rate.
   - **Milestones:** Modify the milestone values for adjusting the learning rate schedule.
   - **Weight Decay:** Explore different weight decay values.
   

3. **Re-Training:** Train the model with the adjusted hyperparameters. Record the training progress and generate a new report, including performance metrics and line plots as before.

4. **Final Analysis:** Analyze the results of the fine-tuned model and compare them with the initial training. Answer the following questions:
   - Has parameter fine-tuning improved model performance?
   - Did it mitigate overfitting or underfitting issues?
   - What can you conclude about the optimal hyperparameters for this task?


---

### **Before parameter fine-tuning**

**Is there evidence of overfitting or underfitting in the initial training results?**

In the initial training results a peculiar behavior happens. It is possible to notice from the graph above that there is a large gap between the training loss and the validation loss with the first one being higher than the latter. Since the model learns from the training set, it should be able to predict the training data better, yet the training loss is much higher than the validation loss. Usually, in the case of underfitting, the loss function is high and doesn’t decrease with the number of iterations both for the validation and training curves or by the training loss that decreases and continues to decrease at the end of the plot. In the case of overfitting the validation loss goes down until a turning point is found, and there it starts going up again. These behaviors don’t match the situation that we experienced so it is quite hard to understand if there is evidence of underfitting or overfitting. 
 
**Are there fluctuations in training and validation loss or accuracy? If so, what might be causing them?**

While the training loss doesn’t look particularly noisy, the validation loss fluctuates especially during the first epochs. Also, this behavior is quite odd and one reason might be that the validation set is widely represented by the training dataset, so the model performs extremely well on the given examples. 

Questo pezzo andrebbe tolto perché non ne siamo certi:
Moreover, a possible reason for the training loss to be higher is that we apply the regularization (using the weight decay) while training the model this increases the training loss. Contrarily, the validation loss only comprises prediction error and this results in a generally lower loss than the training set. 
 
**What can you infer from the initial learning rate, milestones, and weight decay settings?** 

For what we said so far, we may think to slightly lower the regularization (weight decay) in order to understand if we manage to solve the huge gap between the training loss and the validation loss. Moreover, we might want to choose a lower learning rate in order to try to make convergence more linear and possibly increase the generalization accuracy of the model. Concerning the milestones, the learning rate decay should be possibly applied when the training stays at level. In this case, it is firstly applied at the tenth epoch when the training error is still decreasing so it might be an idea to increase the epoch at which it is first applied.

---

### **After parameter fine-tuning**

**Has parameter fine-tuning improved model performance?**

To test the hypotheses mentioned in the previous answer, we have created a new notebook where we performed a gridsearch approach with the following parameter configurations:
-	learning_rates: [1e-1, 1e-2]
-	milestones: [[10, 25, 30], [15, 25, 35], [15, 30]]
-	weight_decays: [0, 1e-2, 1e-5]

After running all the configurations, those that gave us the most satisfactory results were the following:
- learning_rate: 1e-2
- milestones: [15, 25 ,30]
- weight_decay: 1e-2

![tuned](./HyperparamsTuning/plots/loss_epoch_40-0.01-[15,%2025,%2035]-0.01.png)

Compared to the original model which had the following parameters:
- learning_rate: 1e-1
- milestones: [10, 30]
- weight_decay: 1e-05

![image](./checkpoints/plots/loss_epoch_40-0.1-[10,%2030]-1e-05.png)
 
It can be seen that the value of the loss is the same in both the default model and the tuned model, but more importantly, the behaviour of the curves in the tuned model is much smoother, there are no large peaks as in the default model, the descent is much more linear and, as one converges, the difference between the train loss and the validation loss becomes smaller and smaller until it converges towards a possible 'best fit'.

**Did it mitigate overfitting or underfitting issues?**

As mentioned earlier, it is difficult to tell what is happening in the default model because the validation loss lies far below the train loss but they are both stable and do not continue to fall and at the same time we cannot tell if there is overfitting because the valid loss lies far below the train loss. Whereas in the tuned model we have a more optimal situation where the training loss falls as the validation loss with a consequent reduction in the gap between both losses.

**What can you conclude about the optimal hyperparameters for this task?**

In conclusion, it seems that in order to experience a smoother converge a smaller learning rate is needed. Moreover by lowering the regularization (weight decay), we slighty reduced the gap between training loss and validation loss. Concerning the milestones, we increased the epoch at which the learning rate decay it is first applied and also applied another one at the 25th epoch and this allowed to take smaller steps and decrease the loss even more when reaching a stable situation.

## Calculating MPJPE for a Specific Frame  (**2 Points**)


In this exercise, you will calculate the Mean Per Joint Position Error (MPJPE) for a specific frame. This skill is valuable for assessing the accuracy of your model's predictions at a particular moment.

\begin{align*}
\text{MPJPE}_t = \frac{1}{N} \sum_{j=1}^{J} \left\| P_{\text{predicted}_{t,j}} - P_{\text{gt}_{t,j}} \right\|
\end{align*}

Fixed the frame $t$, you will calculate the MPJPE for the predicted pose and the ground truth. Steps:

- Write a function that takes in input the predicted pose and the ground truth one and returns the MPJPE for a number of frames $t$. (e.g. the output could be a dictionary with the frame number as key and the MPJPE as value)
- Rewrite the test function to use the function you just wrote.
- Run the newly created test function for $t=[5, 10, 15, 25]$ and report the results in a table and plot.





In [ ]:
def mpjpe_per_frame(sequences_predict, sequences_gt, frames_to_consider):
    '''
    Insert your code below
    '''
    mpjpe_per_frames = {}

    for f in frames_to_consider:
      #Formula for computing the mpjpe given a prediction and the ground truth
      mpjpe_loss_per_frame = torch.mean(torch.norm(sequences_predict[:, :f].contiguous().view(-1,3) - sequences_gt[:, :f].contiguous().view(-1,3), p=2, dim=1))
      #Add a key-value element in the dict where the key is the number of frame and the value is the associated loss
      mpjpe_per_frames[f] = mpjpe_loss_per_frame.item()

    return mpjpe_per_frames

In [ ]:
#Plot the results for each frame and the average
def plot_mpjpe_results(action, frames_loss, avg_loss):
  frames = [str(key) for key in frames_loss.keys()]
  losses = frames_loss.values()
  plt.bar(frames, losses)
  plt.bar('avg', avg_loss)
  plt.xlabel('Frame')
  plt.ylabel('Loss')
  plt.title(f'Losses for action "{action}"')

  plt.show()

In [ ]:
def test_per_frame(ckpt_path=None):
    model.load_state_dict(torch.load(ckpt_path, map_location=torch.device('cpu')))
    print('model loaded')
    model.eval()
    accum_loss=0
    n_batches=0 # number of batches for all the sequences
    actions=define_actions(actions_to_consider_test)
    dim_used = np.array([6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 21, 22, 23, 24, 25,
                      26, 27, 28, 29, 30, 31, 32, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45,
                      46, 47, 51, 52, 53, 54, 55, 56, 57, 58, 59, 63, 64, 65, 66, 67, 68,
                      75, 76, 77, 78, 79, 80, 81, 82, 83, 87, 88, 89, 90, 91, 92])
    # joints at same loc
    joint_to_ignore = np.array([16, 20, 23, 24, 28, 31])
    index_to_ignore = np.concatenate((joint_to_ignore * 3, joint_to_ignore * 3 + 1, joint_to_ignore * 3 + 2))
    joint_equal = np.array([13, 19, 22, 13, 27, 30])
    index_to_equal = np.concatenate((joint_equal * 3, joint_equal * 3 + 1, joint_equal * 3 + 2))
    totalll=0
    counter=0
    for action in actions:
      running_loss=0
      n=0
      dataset_test = datasets.Datasets(data_path,input_n,output_n,skip_rate, split=2,actions=[action])
      #print('>>> test action for sequences: {:d}'.format(dataset_test.__len__()))

      test_loader = DataLoader(dataset_test, batch_size=batch_size_test, shuffle=False, num_workers=0, pin_memory=True)
      for cnt,batch in enumerate(test_loader):
        with torch.no_grad():

          batch=batch.to(device)
          batch_dim=batch.shape[0]
          n+=batch_dim


          all_joints_seq=batch.clone()[:, input_n:input_n+output_n,:]

          sequences_train=batch[:, 0:input_n, dim_used].view(-1,input_n,len(dim_used)//3,3).permute(0,3,1,2)
          sequences_gt=batch[:, input_n:input_n+output_n, :]


          running_time = time.time()
          sequences_predict=model(sequences_train).view(-1, output_n, joints_to_consider, 3)
          #sequences_predict = model(sequences_train)
          totalll += time.time()-running_time
          counter += 1
          sequences_predict=sequences_predict.contiguous().view(-1,output_n,len(dim_used))

          all_joints_seq[:,:,dim_used] = sequences_predict


          all_joints_seq[:,:,index_to_ignore] = all_joints_seq[:,:,index_to_equal]

          '''
          Insert your code below.
          The function mpjpe_per_frame should return the loss for each frame in the sequence. (e.g. a dictionary with keys the frames and values the loss for each frame)
          Keep a tab of the running loss for each frame and the number of frames in the sequence.
          '''
          frames_to_consider = [5, 10, 15, 25]
          #Compute the mpjpe loss for each specified frame in the sequence
          dict_loss=mpjpe_per_frame(all_joints_seq.view(-1,output_n,32,3), sequences_gt.view(-1,output_n,32,3), frames_to_consider)

      '''
      Insert your code below.
      Average the loss over all the frames in the sequence and print the results.
      '''
      #Compute and print the average
      avg_loss = np.sum(list(dict_loss.values())) / len(dict_loss)
      print(action, dict_loss)
      print(f'Avg loss over all the frames in the sequence for {action}: {avg_loss}')

      #Plot the losses
      plot_mpjpe_results(action, dict_loss, avg_loss)



In [ ]:
'''
Insert your code below where you want to load the model and test it.
You need to specify the path to the model checkpoint file and call the test function.
'''
ckpt_path = './checkpoints/h36m_3d_25frames_ckpt_best_model.pt'
test_per_frame(ckpt_path)

### Table of the losses
Here is a summary table that shows the losses for each combination (action, number of frames):

|                  | 5                  | 10                 | 15                 | 25                 | Avg                |
|------------------|--------------------|--------------------|--------------------|--------------------|--------------------|
| Walking          | 34.629966735839844 | 42.70122528076172  | 46.95539855957031  | 54.42466354370117  | 44.67781352996826  |
| Eating           | 29.550893783569336 | 37.37389373779297  | 47.40604782104492  | 63.45310592651367  | 44.445985317230225 |
| Smoking          | 33.13121795654297  | 43.900936126708984 | 51.43533706665039  | 60.35824203491211  | 47.20643329620361  |
| Discussion       | 39.235172271728516 | 59.414955139160156 | 77.85009765625     | 98.20275115966797  | 68.67574405670166  |
| Greeting         | 72.47596740722656  | 97.70884704589844  | 113.75923919677734 | 147.1549072265625  | 107.77474021911621 |
| Phoning          | 36.620662689208984 | 51.10441970825195  | 63.651424407958984 | 80.80469512939453  | 58.04530048370361  |
| Posing           | 35.54082489013672  | 43.189170837402344 | 53.665687561035156 | 73.08380889892578  | 51.369873046875    |
| Purchases        | 42.522579193115234 | 53.23264694213867  | 59.42998504638672  | 66.7283706665039   | 55.47839546203613  |
| Sitting          | 45.522377014160156 | 58.233375549316406 | 71.22578430175781  | 94.15482330322266  | 67.28409004211426  |
| Sitting down     | 85.18867492675781  | 102.36766815185547 | 124.53817749023438 | 150.81251525878906 | 115.72675895690918 |
| Taking photo     | 46.342864990234375 | 58.62272262573242  | 72.20848846435547  | 92.28362274169922  | 67.36442470550537  |
| Waiting          | 38.32765579223633  | 51.2755012512207   | 63.136619567871094 | 79.09365844726562  | 57.95835876464844  |
| Walking dog      | 41.116111755371094 | 45.98772430419922  | 51.71000671386719  | 65.6707763671875   | 51.12115478515625  |
| Walking together | 39.820701599121094 | 48.1856689453125   | 55.00767517089844  | 62.043296813964844 | 51.26433563232422  |

## Iterative Mechanism (**3 Points**)

In this exercise, you will explore the concept of an iterative mechanism and its adaptability when the model's output length changes. You will start with a model designed to produce 25 output frames but adapt it to generate only 10. The exercise will involve modifying and re-training the model for the new output length. During test time, the model will generate 10 frames and then use them as input to generate the successive 10 frames, and so on, until the desired number of frames is reached. In this case, you are asked to generate 25 frames.

The steps are as follows:
- Change the model's output length from 25 to 10.
- Re-train the model.
- Rewrite the test function to generate 25 frames using the iterative mechanism.
- Generate a new report and compare the results to the baseline model.

In [ ]:
import os
# # Arguments to setup the datasets
datas = 'h36m' # dataset name
data_path = './data/h3.6m/h3.6m/dataset/'
input_n=10 # number of frames to train on (default=10)

'''
Insert your code below
'''
output_n= 10

input_dim=3 # dimensions of the input coordinates(default=3)
skip_rate=1 # # skip rate of frames
joints_to_consider=22

#FLAGS FOR THE TRAINING
mode='train' #choose either train or test mode

batch_size_test=8
checkpoint_it_path = './checkpoints_it/' # path to the model checkpoint directory
#model_path_iterative= os.path.join(checkpoint_path, sorted(os.listdir(checkpoint_path))[-1]) # path to the latest model checkpoint file

actions_to_consider_test='all' # actions to test on.
model_name = datas+'_3d_'+str(output_n)+'frames_ckpt' #the model name to save/load

#FLAGS FOR THE VISUALIZATION
actions_to_consider_viz='all' # actions to visualize
visualize_from='test'
n_viz=2

In [ ]:
# Load Data
print('Loading Train Dataset...')
dataset = datasets.Datasets(data_path,input_n,output_n,skip_rate, split=0)
print('Loading Validation Dataset...')
vald_dataset = datasets.Datasets(data_path,input_n,output_n,skip_rate, split=1)

#! Note: Ignore warning:  "VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences"

In [ ]:
batch_size=256

print('>>> Training dataset length: {:d}'.format(dataset.__len__()))
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)#

print('>>> Validation dataset length: {:d}'.format(vald_dataset.__len__()))
vald_loader = DataLoader(vald_dataset, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)

In [ ]:
from models.sttr.sttformer import Model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device: %s'%device)

n_heads = 1

model = Model(num_joints=joints_to_consider,
                 num_frames=input_n, num_frames_out=output_n, num_heads=n_heads,
                 num_channels=3, kernel_size=[3,3], use_pes=True).to(device)

print('total number of parameters of the network is: '+str(sum(p.numel() for p in model.parameters() if p.requires_grad)))

In [ ]:
# Arguments to setup the optimizer
lr=1e-01 # learning rate
use_scheduler=True # use MultiStepLR scheduler
milestones=[10,30]   # the epochs after which the learning rate is adjusted by gamma
gamma=0.1 #gamma correction to the learning rate, after reaching the milestone epochs
weight_decay=1e-05 # weight decay (L2 penalty)
optimizer=optim.Adam(model.parameters(),lr=lr,weight_decay=weight_decay)

if use_scheduler:
    scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=milestones, gamma=gamma)

clip_grad=None # select max norm to clip gradients
# Argument for training
n_epochs=41
log_step = 200

### Train and Validation Loop

In [ ]:
save_and_plot = True # save the model and plot the loss. Change to True if you want to save the model and plot the loss

In [ ]:
from os import path
if path.exists("./checkpoints_it/") == False:
    os.mkdir("./checkpoints_it/")

# launch training with the new output_n
train(data_loader,vald_loader, path_to_save_model=checkpoint_it_path)

### Test Loop

In [ ]:
def test(ckpt_path=None):
    with open (ckpt_path, 'rb') as f:
        model.load_state_dict(torch.load(f, map_location=device))
    print('model loaded')
    model.eval()
    accum_loss=0
    n_batches=0 # number of batches for all the sequences
    actions=define_actions(actions_to_consider_test)
    dim_used = np.array([6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 21, 22, 23, 24, 25,
                      26, 27, 28, 29, 30, 31, 32, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45,
                      46, 47, 51, 52, 53, 54, 55, 56, 57, 58, 59, 63, 64, 65, 66, 67, 68,
                      75, 76, 77, 78, 79, 80, 81, 82, 83, 87, 88, 89, 90, 91, 92])
    # joints at same loc
    joint_to_ignore = np.array([16, 20, 23, 24, 28, 31])
    index_to_ignore = np.concatenate((joint_to_ignore * 3, joint_to_ignore * 3 + 1, joint_to_ignore * 3 + 2))
    joint_equal = np.array([13, 19, 22, 13, 27, 30])
    index_to_equal = np.concatenate((joint_equal * 3, joint_equal * 3 + 1, joint_equal * 3 + 2))
    totalll=0
    counter=0
    for action in actions:
      running_loss=0
      n=0

      FINAL_SEQUENCE_LENGTH = 25

      dataset_test = datasets.Datasets(data_path,input_n,FINAL_SEQUENCE_LENGTH,skip_rate, split=2,actions=[action])
      #print('>>> test action for sequences: {:d}'.format(dataset_test.__len__()))
      test_loader = DataLoader(dataset_test, batch_size=batch_size_test, shuffle=False, num_workers=0, pin_memory=True)

      for cnt, batch in enumerate(test_loader):
        final_predicted_sequence = None
        with torch.no_grad():

          batch=batch.to(device)
          batch_dim=batch.shape[0]
          n+=batch_dim

          # ground truth and the final sequence length should be 25, even if the output of the model is 10.
          all_joints_seq=batch.clone()[:, input_n:input_n+FINAL_SEQUENCE_LENGTH,:]
          sequences_gt = batch[:, input_n:input_n+FINAL_SEQUENCE_LENGTH, :]

          running_time = time.time()
          '''
          Insert your code below. You will need to iteratively predict the next frames and feed it to back to the model until you reach the desired number of frames.
          '''
          # Getting the current sequence from the batch (length 10)
          sequences_predict = batch[:, 0:input_n, dim_used]
          # Until I get the desired sequence length
          while final_predicted_sequence is None or final_predicted_sequence.shape[1] < FINAL_SEQUENCE_LENGTH:
            # I reshape the sequence to feed it to the model
            sequences_predict = sequences_predict.view(-1,input_n,len(dim_used)//3,3).permute(0,3,1,2)
            sequences_predict = model(sequences_predict)

            # I keep another copy of the sequence reshaped in another way in order to concatenate it to the final sequence
            reshaped_sequences_predict = sequences_predict.view(-1, output_n, joints_to_consider, 3)


            if final_predicted_sequence is None:
              # If the final sequence is empty, I just assign the first sequence predicted
              final_predicted_sequence = reshaped_sequences_predict
            else:
              # Otherwise I concatenate the new sequence to the final one, checking that the final sequence length is not greater than 25
              if (final_predicted_sequence.shape[1] + sequences_predict.shape[1] > FINAL_SEQUENCE_LENGTH):
                reshaped_sequences_predict = reshaped_sequences_predict[:, :reshaped_sequences_predict.shape[1] + output_n - FINAL_SEQUENCE_LENGTH, :, :]
              final_predicted_sequence = torch.cat((final_predicted_sequence, reshaped_sequences_predict), dim=1)

            totalll += time.time()-running_time
            counter += 1

          # Trick to get represent the final sequence in the same way as the ground truth (32 joints instead of 22)
          all_joints_seq[:,:,dim_used] = final_predicted_sequence.contiguous().view(-1,FINAL_SEQUENCE_LENGTH,len(dim_used))
          all_joints_seq[:,:,index_to_ignore] = all_joints_seq[:,:,index_to_equal]

          all_joints_seq = all_joints_seq.view(-1, FINAL_SEQUENCE_LENGTH, 32, 3)
          sequences_gt = sequences_gt.view(-1, FINAL_SEQUENCE_LENGTH, 32, 3)

          loss = mpjpe_error(all_joints_seq,sequences_gt)
          running_loss+=loss*batch_dim
          accum_loss+=loss*batch_dim

      #print('loss at test subject for action : '+str(action)+ ' is: '+ str(running_loss/n))
      print(str(action),': ', str(np.round((running_loss/n).item(),1)))
      n_batches+=n
    print('Average: '+str(np.round((accum_loss/n_batches).item(),1)))
    print('Prediction time: ', totalll/counter)

In [ ]:
test(ckpt_path=checkpoint_it_path + 'h36m_3d_25frames_ckpt_best_model.pt')

### Results comparison

For the table below, it is possible to notice how the results of the baseline model and the iterative ones are mostly the same with the baseline model having a slighly lower average loss as well as a lower pradiction time maybe given by the fact that the next frames are already given and they don't have to be predicted:

|                  | Baseline model       | Iterative Model     |
|------------------|----------------------|---------------------|
| Walking          | 59.9                 | 74.7                |
| Eating           | 58.3                 | 61.8                |
| Smoking          | 60.2                 | 88.7                |
| Discussion       | 86.4                 | 78.9                |
| Greeting         | 77,6                 | 104.0               |
| Phoning          | 100.7                | 76.9                |
| Posing           | 113.8                | 114.3               |
| Purchases        | 102.6                | 101.2               |
| Sitting          | 88.1                 | 87.4                |
| Sitting down     | 109.6                | 114.7               |
| Taking photo     | 85.1                 | 88.4                |
| Waiting          | 81.4                 | 82.3                |
| Walking dog      | 110.6                | 111.3               |
| Walking together | 58.6                 | 67.5                |
| AVG              | 84.5                 | 87.6                |
| Prediction time  | 0.016339437663555147 | 0.04167355961269802 |

## YOUR custom model (**3 Points**)
 In this exercise, you will implement a Transformer-like network (based on the Theory notebook) for this specific task. You can use the Transformer's Encoder and implement your own Decoder to predict future poses. (e.g. RNN, MLP, CNN, TCN, ...).
 We won't provide any code for this exercise, but you can use the code provided in the Theory notebook as a starting point.
 The goal of this exercise is not to beat the previous model but to understand how to implement a Transformer network for this specific task. For this reason, the evaluation will be based on the code you write and the explanation you provide in the report rather than the results.


### Performance BONUS (**Up to 2 Points**)
- **1 Bonus Point** if the model achieves an A-MPJPE between 80 and 90 millimeters.
- **2 Bonus Points** if the model achieves an A-MPJPE between 70 and 80 millimeters.

## Report Results
Initially, to get an idea, we tried the standard Transformer, implemented in the theory part, with the addition of the iterative mechanism. Unfortunately it performed poorly with a minimum validation loss of 240,
To understand if it was the decoder mechanism or it was all the architecture which was not good for the task, we experimented with only the encoder part plus a final conv 1D layer that augment the frames from 10 to 25, but the performances remain the same.
The next move was to augment the capacities of the transformer with convolutions and long-short term memory layers, the idea was to firstly extract spatial features with convolutions, next temporal features with LSTM and only at that point take advantage of the self-attention mechanism.
This model is represented by the class First_Model.
The metrics, even though slighlty improved, were still disappointing (val loss 200).

At this point we decided to merge the STTformer with the standard Transformer, using the STAblock in the encoder part and the more classical decoder multi-heads masking self-attention followed by the encoder-decoder attention (K and V of the encoder output and Q of the first decoder layer output). To match the number of frames in input with the the number of frames that we have to predict we rely on the iterative mechanism.
This model is represented by the class Second_Model.
With this model we reach a test loss of (A-MPJPE error) *50* and a val loss of val loss:  *78.83*.
In conclusion the final solution is a sort of merge with all the architectures and mechanisms learned: self-attention, encoder-decoder, iterative mechanism, STAblock, positional encoding and masking.


In [ ]:
'''
Input Shape: [batch_size, input_time, joints, 3]

Encoder:
Input shape: [batch_size, input_time, joints, 3]
Output shape: [batch_size, input_time/output_time, joints, FREE]

# Decoder:
Input shape: [batch_size, input_time/output_time, joints, FREE]
Output shape: [batch_size, output_time, joints, 3]


Hint: Transformers often take an input of shape [batch_size, time, joints*channels], use the reshape or view function to match the dimensionality.
'''

In [ ]:
# # Arguments to setup the datasets
datas = 'h36m' # dataset name
data_path = './data/h3.6m/h3.6m/dataset'
input_n=10 # number of frames to train on (default=10)
output_n=25 # number of frames to predict on
input_dim=3 # dimensions of the input coordinates(default=3)
skip_rate=1 # # skip rate of frames
joints_to_consider=22


#FLAGS FOR THE TRAINING
mode='train' #choose either train or test mode

batch_size_test=8
checkpoint_our_model_path = './checkpoints_ourmodel/' # path to the model checkpoint directory

actions_to_consider_test='all' # actions to test on.
model_name = datas+'_3d_'+str(output_n)+'frames_ckpt' #the model name to save/load

#FLAGS FOR THE VISUALIZATION
actions_to_consider_viz='all' # actions to visualize
visualize_from='test'
n_viz=2

In [ ]:
# Load Data
print('Loading Train Dataset...')
dataset = datasets.Datasets(data_path,input_n,output_n,skip_rate, split=0)
print('Loading Validation Dataset...')
vald_dataset = datasets.Datasets(data_path,input_n,output_n,skip_rate, split=1)

#! Note: Ignore warning:  "VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences"

In [ ]:
batch_size=256

print('>>> Training dataset length: {:d}'.format(dataset.__len__()))
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)#

print('>>> Validation dataset length: {:d}'.format(vald_dataset.__len__()))
vald_loader = DataLoader(vald_dataset, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)

In [ ]:
from models.sttr.sttformer import Model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device: %s'%device)

n_heads = 1

model = Model(num_joints=joints_to_consider,
                 num_frames=input_n, num_frames_out=output_n, num_heads=n_heads,
                 num_channels=3, kernel_size=[3,3], use_pes=True).to(device)

print('total number of parameters of the network is: '+str(sum(p.numel() for p in model.parameters() if p.requires_grad)))

In [ ]:
from torch.autograd import Variable
import math
from torch import nn

class Attention(nn.Module):
    ''' Scaled Dot-Product Attention '''

    def __init__(self, attn_dropout=0.1):
        super().__init__()
        self.dropout = nn.Dropout(attn_dropout)

    def forward(self, query, key, value, mask=None):

        d_k = query.size(-1)  # Get the size of the key
        attn = torch.einsum('BHLD, BHMD -> BHLM', query, key)  # Compute the dot product of the query and key, and scale it

        if mask is not None:
            attn = attn.masked_fill(mask == 0, -1e9)

        attn = self.dropout(F.softmax(attn/math.sqrt(d_k), dim=-1))

        output = torch.einsum('BHLL, BHLD -> BHLD', attn, value)  # Compute the weighted sum of the values

        return output, attn

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

class EncoderLayer(nn.Module):
    ''' Compose with two layers '''

    def __init__(self, d_model, d_inner, n_head, d_k, d_v, dropout=0.1):
        super(EncoderLayer, self).__init__()
        self.slf_attn = MultiHeadAttention(n_head, d_model, d_k, d_v, dropout=dropout)
        self.pos_ffn = PositionwiseFeedForward(d_model, d_inner, dropout=dropout)

    def forward(self, enc_input, slf_attn_mask=None):
        enc_output, enc_slf_attn = self.slf_attn(
            enc_input, enc_input, enc_input, mask=slf_attn_mask)
        enc_output = self.pos_ffn(enc_output)
        return enc_output, enc_slf_attn

class DecoderLayer(nn.Module):
    ''' Compose with three layers '''

    def __init__(self, d_model, d_inner, n_head, d_k, d_v, dropout=0.1):
        super(DecoderLayer, self).__init__()
        self.slf_attn = MultiHeadAttention(n_head, d_model, d_k, d_v, dropout=dropout)
        self.enc_attn = MultiHeadAttention(n_head, d_model, d_k, d_v, dropout=dropout)
        self.pos_ffn = PositionwiseFeedForward(d_model, d_inner, dropout=dropout)

    def forward(
            self, dec_input, enc_output,
            slf_attn_mask=None, dec_enc_attn_mask=None):
        dec_output, dec_slf_attn = self.slf_attn(
            dec_input, dec_input, dec_input, mask=slf_attn_mask)
        dec_output, dec_enc_attn = self.enc_attn(
            dec_output, enc_output, enc_output, mask=dec_enc_attn_mask)
        dec_output = self.pos_ffn(dec_output)
        return dec_output, dec_slf_attn, dec_enc_attn

class MultiHeadAttention(nn.Module):
    ''' Multi-Head Attention module '''

    def __init__(self, n_head, d_model, d_k, d_v, dropout=0.1):
        super().__init__()

        self.n_head = n_head
        self.d_k = d_k
        self.d_v = d_v

        self.w_qs = nn.Linear(d_model, n_head * d_k, bias=False)
        self.w_ks = nn.Linear(d_model, n_head * d_k, bias=False)
        self.w_vs = nn.Linear(d_model, n_head * d_v, bias=False)
        self.fc = nn.Linear(n_head * d_v, d_model, bias=False)

        self.attention = Attention()

        self.dropout = nn.Dropout(dropout)
        self.layer_norm = nn.LayerNorm(d_model, eps=1e-6)


    def forward(self, q, k, v, mask=None):

        d_k, d_v, n_head = self.d_k, self.d_v, self.n_head
        sz_b, len_q, len_k, len_v = q.size(0), q.size(1), k.size(1), v.size(1)

        residual = q

        # Pass through the pre-attention projection: b x lq x (n*dv)
        # Separate different heads: b x lq x n x dv
        q = self.w_qs(q).view(sz_b, len_q, n_head, d_k)
        k = self.w_ks(k).view(sz_b, len_k, n_head, d_k)
        v = self.w_vs(v).view(sz_b, len_v, n_head, d_v)

        # Transpose for attention dot product: b x n x lq x dv
        q, k, v = q.transpose(1, 2), k.transpose(1, 2), v.transpose(1, 2)

        if mask is not None:
            mask = mask.unsqueeze(1)   # For head axis broadcasting.

        q, attn = self.attention(q, k, v, mask=mask)

        # Transpose to move the head dimension back: b x lq x n x dv
        # Combine the last two dimensions to concatenate all the heads together: b x lq x (n*dv)
        q = q.transpose(1, 2).contiguous().view(sz_b, len_q, -1)
        q = self.dropout(self.fc(q))
        q += residual

        q = self.layer_norm(q)

        return q, attn


class PositionwiseFeedForward(nn.Module):

    def __init__(self, d_in, d_hidden, dropout=0.1):
        super().__init__()
        self.w1 = nn.Linear(d_in, d_hidden) # position-wise
        self.w2 = nn.Linear(d_hidden, d_in) # position-wise
        self.layer_norm = nn.LayerNorm(d_in, eps=1e-6)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):

        residual = x

        x = self.w2(F.relu(self.w1(x)))
        x = self.dropout(x)
        x += residual

        x = self.layer_norm(x)

        return x

def get_pad_mask(seq, pad_idx):
    return (seq != pad_idx).unsqueeze(-2)


def get_subsequent_mask(seq):
    ''' For masking out the subsequent info. '''
    if (len(seq.shape) == 4):
      sz_b, c, len_s, dim = seq.size()
    else:
      sz_b, len_s, dim = seq.size()
    subsequent_mask = (1 - torch.triu(
        torch.ones((1, len_s, len_s), device=seq.device), diagonal=1)).bool()
    return subsequent_mask


class Encoder(nn.Module):
    ''' A encoder model with self attention mechanism. '''

    def __init__(
            self, n_layers, n_head, d_k, d_v,
            d_model, d_inner, pad_idx, dropout=0.1, n_position=200, scale_emb=False):

        super().__init__()


        self.position_enc = PositionalEncoding(d_model)
        self.dropout = nn.Dropout(p=dropout)
        self.layer_stack = nn.ModuleList([
            EncoderLayer(d_model, d_inner, n_head, d_k, d_v, dropout=dropout)
            for _ in range(n_layers)])
        self.layer_norm = nn.LayerNorm(d_model, eps=1e-6)
        self.scale_emb = scale_emb
        self.d_model = d_model

    def forward(self, src_seq, src_mask, return_attns=False):

        enc_slf_attn_list = []

        enc_output = self.dropout(self.position_enc(src_seq))
        enc_output = self.layer_norm(enc_output)

        for enc_layer in self.layer_stack:
            enc_output, enc_slf_attn = enc_layer(enc_output, slf_attn_mask=src_mask)
            enc_slf_attn_list += [enc_slf_attn] if return_attns else []

        if return_attns:
            return enc_output, enc_slf_attn_list
        return enc_output,

class Decoder(nn.Module):
    ''' A decoder model with self attention mechanism. '''

    def __init__(
            self, d_word_vec, n_layers, n_head, d_k, d_v,
            d_model, d_inner, pad_idx, n_position=200, dropout=0.1, scale_emb=False):

        super().__init__()

        self.position_enc = PositionalEncoding(d_word_vec)
        self.dropout = nn.Dropout(p=dropout)
        self.layer_stack = nn.ModuleList([
            DecoderLayer(d_model, d_inner, n_head, d_k, d_v, dropout=dropout)
            for _ in range(n_layers)])
        self.layer_norm = nn.LayerNorm(d_model, eps=1e-6)
        self.scale_emb = scale_emb
        self.d_model = d_model

    def forward(self, trg_seq, trg_mask, enc_output, src_mask, inference):
        if not inference:
          dec_output = self.dropout(self.position_enc(trg_seq))
          dec_output = self.layer_norm(dec_output)

          for dec_layer in self.layer_stack:
              dec_output, dec_slf_attn, dec_enc_attn = dec_layer(
                  dec_output, enc_output, slf_attn_mask=trg_mask, dec_enc_attn_mask=src_mask)
          return dec_output

        else:
          for i in range(trg_seq.shape[1]):
            #print(trg_seq[0, :, 3])
            if i!= trg_seq.shape[1]-1:
              dec_output = self.dropout(self.position_enc(trg_seq))
              dec_output = self.layer_norm(dec_output)

              for dec_layer in self.layer_stack:
                  dec_output, dec_slf_attn, dec_enc_attn = dec_layer(
                      dec_output, enc_output, slf_attn_mask=trg_mask, dec_enc_attn_mask=src_mask)
              trg_seq[:, i+1, :] = dec_output[:, -1, :]
            else:
              dec_output = self.dropout(self.position_enc(trg_seq))
              dec_output = self.layer_norm(dec_output)

              for dec_layer in self.layer_stack:
                  dec_output, dec_slf_attn, dec_enc_attn = dec_layer(
                      dec_output, enc_output, slf_attn_mask=trg_mask, dec_enc_attn_mask=src_mask)

          return torch.cat((trg_seq[:, 1:, :], dec_output[:, -1, :].unsqueeze(1)), dim=1)

class First_Model(nn.Module):
    ''' A sequence to sequence model with attention mechanism. '''

    def __init__(
            self, src_pad_idx, trg_pad_idx,
            d_model,
            n_layers, n_head,dropout=0.1, n_position=200,
            trg_emb_prj_weight_sharing=True, emb_src_trg_weight_sharing=True,
            scale_emb_or_prj='prj'):

        super().__init__()
        d_inner = d_model * 4
        d_k = d_v = d_model // n_head
        self.src_pad_idx, self.trg_pad_idx = src_pad_idx, trg_pad_idx

        scale_emb = (scale_emb_or_prj == 'emb') if trg_emb_prj_weight_sharing else False
        self.scale_prj = (scale_emb_or_prj == 'prj') if trg_emb_prj_weight_sharing else False
        self.d_model = d_model

        self.encoder = Encoder(
            n_position=n_position,
             d_model=d_model, d_inner=d_inner,
            n_layers=n_layers, n_head=n_head, d_k=d_k, d_v=d_v,
            pad_idx=src_pad_idx, dropout=dropout, scale_emb=scale_emb)

        self.decoder = Decoder(
            n_position=n_position,
            d_word_vec=d_word_vec, d_model=d_model, d_inner=d_inner,
            n_layers=n_layers, n_head=n_head, d_k=d_k, d_v=d_v,
            pad_idx=trg_pad_idx, dropout=dropout, scale_emb=scale_emb)

        self.conv_out = nn.Conv1d(in_channels=10, out_channels=25, kernel_size=1, stride=1)

        self.conv_in1 = nn.Conv2d(3, 16, (3,3), padding='same')
        self.conv_in2 = nn.Conv2d(16, 16, (5,5), padding='same')
        self.conv_in3 = nn.Conv2d(16, 16, (7,7), padding='same')

        self.lstm = nn.LSTM(22*16, d_model, batch_first=True)
        self.lstm_out = nn.LSTM(d_model, 66, batch_first=True)
        self.lrelu = nn.LeakyReLU(inplace=True)
        for p in self.parameters():
            if p.dim() > 1:
                nn.init.xavier_uniform_(p)

    def forward(self, src_seq, trg_seq):
        src_mask = None
        trg_mask = get_subsequent_mask(trg_seq)

        src_seq = self.lrelu(self.conv_in1(src_seq))

        src_seq = self.lrelu(self.conv_in2(src_seq)+src_seq)

        src_seq = self.lrelu(self.conv_in3(src_seq)+src_seq)

        src_seq = src_seq.reshape(-1, src_seq.shape[2], src_seq.shape[1]*src_seq.shape[3])

        src_seq, (_, _) = self.lstm(src_seq)

        enc_output, *_ = self.encoder(src_seq, src_mask)
        dec_output, *_ = self.decoder(trg_seq, trg_mask, enc_output, src_mask)
        dec_output, (_, _) = self.lstm_out(dec_output)
        dec_output = self.conv_out(dec_output)
        return dec_output

input_size = 66
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_heads = 6
d_model = 128
dropout = 0.1

In [ ]:
class PosEmb(nn.Module):
    def __init__(self, d_model: int, channels: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)
        self.channels = channels
        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x):
        """
        Arguments:
            x: Tensor, shape ``[seq_len, batch_size, embedding_dim]``
        """
        for c in range(self.channels):
          x[:, c, :, :] = x[:, c, :, :] + self.pe[:x.size(0)]
        return self.dropout(x)

class STA_Block(nn.Module):
    def __init__(self, in_channels, out_channels, qkv_dim,
                 num_joints, num_heads,
                 kernel_size, use_pes=True, att_drop=0):
        super().__init__()
        self.qkv_dim = qkv_dim
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.num_heads = num_heads
        self.use_pes = use_pes
        pads = int((kernel_size[1] - 1) / 2)
        padt = int((kernel_size[0] - 1) / 2)

        # Spatio-Temporal Tuples Attention
        if self.use_pes: self.pes = PosEmb(num_joints, in_channels)
        self.to_qkvs = nn.Conv2d(in_channels, 2 * num_heads * qkv_dim, 1, bias=True)
        self.alphas = nn.Parameter(torch.ones(1, num_heads, 1, 1), requires_grad=True)
        self.att0s = nn.Parameter(torch.ones(1, num_heads, num_joints, num_joints) / num_joints, requires_grad=True)
        self.out_nets = nn.Sequential(nn.Conv2d(in_channels * num_heads, out_channels, (1, kernel_size[1]), padding=(0, pads)), nn.BatchNorm2d(out_channels))
        self.ff_net = nn.Sequential(nn.Conv2d(out_channels, out_channels, 1), nn.BatchNorm2d(out_channels))

        # Inter-Frame Feature Aggregation
        self.out_nett = nn.Sequential(nn.Conv2d(out_channels, out_channels, (kernel_size[0], 1), padding=(padt, 0)), nn.BatchNorm2d(out_channels))

        if in_channels != out_channels:
            self.ress = nn.Sequential(nn.Conv2d(in_channels, out_channels, 1), nn.BatchNorm2d(out_channels))
            self.rest = nn.Sequential(nn.Conv2d(out_channels, out_channels, 1), nn.BatchNorm2d(out_channels))
        else:
            self.ress = lambda x: x
            self.rest = lambda x: x

        self.tan = nn.Tanh()
        self.relu = nn.LeakyReLU(0.1)
        self.drop = nn.Dropout(att_drop)

    def forward(self, x):

        N, C, T, V = x.size()
        # Spatio-Temporal Tuples Attention
        xs = self.pes(x)
        q, k = torch.chunk(self.to_qkvs(xs).view(N, 2 * self.num_heads, self.qkv_dim, T, V), 2, dim=1)
        attention = self.tan(torch.einsum('nhctu,nhctv->nhuv', [q, k]) / (self.qkv_dim * T)) * self.alphas
        attention = attention + self.att0s.repeat(N, 1, 1, 1)
        attention = self.drop(attention)
        xs = torch.einsum('nctu,nhuv->nhctv', [x, attention]).contiguous().view(N, self.num_heads * self.in_channels, T, V)
        x_ress = self.ress(x)
        xs = self.relu(self.out_nets(xs) + x_ress)
        xs = self.relu(self.ff_net(xs) + x_ress)

        # Inter-Frame Feature Aggregation
        xt = self.relu(self.out_nett(xs) + self.rest(xs))

        return xt

class Second_Model(nn.Module):
    def __init__(self, num_joints, n_layers,
                 num_frames, num_frames_out, num_heads, in_channels, hidden_channels,
                 kernel_size, batch_size, len_parts=1, use_pes=True, config=None, num_persons=1,
                 att_drop=0, dropout=0, dropout2d=0):
        super().__init__()
        self.batch_size = batch_size
        self.num_frames = num_frames
        self.num_joints = num_joints
        self.num_persons = num_persons
        self.len_parts = len_parts
        self.in_channels = in_channels
        self.hidden_channels = hidden_channels

        num_frames = num_frames // len_parts
        num_joints = num_joints * len_parts

        self.input_map = nn.Sequential(
            nn.Conv2d(in_channels, hidden_channels, 1),
            nn.BatchNorm2d(hidden_channels),
            nn.LeakyReLU(0.1))

        self.enc_blocks = nn.ModuleList()
        for index in range(n_layers):
          if (index != n_layers-1):
            self.enc_blocks.append(STA_Block(hidden_channels, hidden_channels, hidden_channels,
                                         num_joints=num_joints,
                                         num_heads=num_heads,
                                         kernel_size=kernel_size,
                                         use_pes=use_pes,
                                         att_drop=att_drop))
          else:
            self.enc_blocks.append(STA_Block(hidden_channels, in_channels, hidden_channels,
                                         num_joints=num_joints,
                                         num_heads=num_heads,
                                         kernel_size=kernel_size,
                                         use_pes=use_pes,
                                         att_drop=att_drop))

        d_model = num_joints * in_channels
        self.decoder = Decoder(d_model, n_layers, num_heads, d_model, d_model, d_model, d_model*4, None)

        self.fc_out = nn.Linear(66, 66)
        self.conv_out = nn.Conv1d(num_frames, num_frames_out, 1, stride=1)


    def forward(self, x, tgt, inference):

        x = x.reshape(self.batch_size, -1, self.num_joints, self.in_channels).permute(0, 3, 1, 2).contiguous()
        N, C, T, V = x.shape

        x = x.view(x.size(0), x.size(1), T // self.len_parts, V * self.len_parts)
        x = self.input_map(x)

        for i, enc_block in enumerate(self.enc_blocks):
            x = enc_block(x)
        x = x.reshape(x.shape[0], x.shape[2], x.shape[1]*x.shape[3])
        mask = get_subsequent_mask(tgt)
        #print("before decoder: ", x.shape)
        x = self.decoder(tgt, mask, x, None, inference)

        #print("after decoder: ", x.shape)
        out = self.fc_out(x) #self.conv_out(x))

        return out

#model = First_Model(0, 0, d_model=d_model, n_layers=6, n_head=num_heads, dropout=dropout).to(device)
model = Second_Model(num_joints=joints_to_consider, n_layers=6,
                 num_frames=10, num_frames_out=10, num_heads=num_heads,
                 in_channels=3, hidden_channels=16, kernel_size=[3,3], batch_size=batch_size, use_pes=True).to(device)


In [ ]:
# Arguments to setup the optimizer
lr=0.01
n_epochs=40 # number of epochs
weight_decay=1e-6 # weight decay (L2 penalty)
optimizer=optim.Adam(model.parameters(),lr=lr,weight_decay=weight_decay)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=n_epochs, eta_min=0.000001)

In [ ]:
import os
from os import path
import copy
flag = True
log_step = 200
def train_custom_model(data_loader,vald_loader, path_to_save_model=None):
  # Creation of a folder where to save plots
  if path.exists(path_to_save_model + "plots/") == False:
    os.mkdir(path_to_save_model + "plots/")

  train_loss = []
  val_loss = []
  val_loss_best = 1000

  dim_used = np.array([6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 21, 22, 23, 24, 25,
                    26, 27, 28, 29, 30, 31, 32, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45,
                    46, 47, 51, 52, 53, 54, 55, 56, 57, 58, 59, 63, 64, 65, 66, 67, 68,
                    75, 76, 77, 78, 79, 80, 81, 82, 83, 87, 88, 89, 90, 91, 92])

  for epoch in range(n_epochs-1):
      running_loss=0
      n=0


      model.train()

      for cnt,batch in enumerate(data_loader):

          batch=batch.float().to(device)
          batch_dim=batch.shape[0]
          if (batch_dim != batch_size):
            continue
          n+=batch_dim
          sequences_train=batch[:, 0:input_n, dim_used]#.view(-1,input_n,len(dim_used)//3,3).permute(0,3,1,2)
          sequences_gt=batch[:, input_n:input_n+output_n, dim_used]#.view(-1,output_n,len(dim_used)//3,3)

          #print(sequences_gt.shape)
          optimizer.zero_grad()
          #print(sequences_gt.shape)
          #print(sequences_train.shape)

          #padding
          #sequences_train = torch.cat((sequences_train, torch.zeros(batch_dim, output_n-input_n, dim_used)).to(device), dim=1)

          start_of_seq = torch.Tensor([1]).repeat(sequences_gt.shape[0], 1, sequences_gt.shape[2]).to(device)
          #print(start_of_seq.shape)
          # Final decoder input is the concatenation of them along temporal dimension
          dec_inp = torch.cat((start_of_seq, sequences_gt), 1)
          dec_inp1 = dec_inp[:, :10, :]
          sequences_predict1=model(sequences_train, dec_inp1, False)
          dec_inp2 = dec_inp[:, 10:20, :]
          sequences_predict2=model(dec_inp1, dec_inp2, False)
          dec_inp3 = dec_inp[:, 20:, :]
          sequences_predict3=model(dec_inp2[:, -6:, :], dec_inp3, False)
          sequences_predict=torch.cat((sequences_predict1,sequences_predict2,sequences_predict3[:, :-1, :]),1)
          #print(sequences_predict.shape)
          #sequences_predict=model(sequences_train, sequences_gt).view(-1, output_n, joints_to_consider, 3)
          loss=mpjpe_error(sequences_predict,sequences_gt)

          if cnt % log_step == 0:
            print('[Epoch: %d, Iteration: %5d]  training loss: %.3f' %(epoch + 1, cnt + 1, loss.item()))

          loss.backward()
          optimizer.step()
          running_loss += loss.item()

      train_loss.append(running_loss/(n//batch_dim))

      model.eval()

      with torch.no_grad():
          running_loss=0
          n=0
          for cnt,batch in enumerate(vald_loader):
              batch=batch.float().to(device)
              batch_dim=batch.shape[0]
              if (batch_dim != batch_size):
                continue
              n+=batch_dim

              sequences_train=batch[:, 0:input_n, dim_used]#.view(-1,input_n,len(dim_used)//3,3).permute(0,3,1,2)
              sequences_gt=batch[:, input_n:input_n+output_n, dim_used]#.view(-1,output_n,len(dim_used)//3,3)

              #print(sequences_gt.shape)
              optimizer.zero_grad()
              #print(sequences_gt.shape)
              #print(sequences_train.shape)
              empty_seq = torch.zeros([sequences_train.shape[0], sequences_train.shape[1]-1, sequences_train.shape[2]]).to(device)
              start_of_seq = torch.Tensor([1]).repeat(sequences_gt.shape[0], 1, sequences_gt.shape[2]).to(device)
              #print(start_of_seq.shape)
              dec_input = torch.cat((start_of_seq, empty_seq), dim=1)
              # Final decoder input is the concatenation of them along temporal dimension

              sequences_predict1=model(sequences_train, dec_input, True)
              sequences_predict2=model(sequences_predict1, dec_input, True)
              sequences_predict3=model(sequences_predict2[:, -6:, :], dec_input[:, :6, :], True)

              sequences_predict=torch.cat((sequences_predict1,sequences_predict2,sequences_predict3[:, :-1, :]),1)

              #sequences_predict=model(sequences_train, sequences_gt).view(-1, output_n, joints_to_consider, 3)
              loss=mpjpe_error(sequences_predict,sequences_gt)
              running_loss += loss.item()

          running_loss = running_loss / (n//batch_dim)
          print("val loss: ", running_loss)
          val_loss.append(running_loss)
          if running_loss/n < val_loss_best:
            val_loss_best = running_loss/n
            best_model = copy.deepcopy(model)

      scheduler.step()
      print(scheduler.get_lr())
      # save and plot model every 5 epochs
      '''
      Insert your code below. Use the argument path_to_save_model to save the model to the path specified.
      '''
      if save_and_plot and (epoch+1)%5==0 and epoch!=0: # Save the model and display the losses every 5 epochs
        torch.save(model.state_dict(), f'{path_to_save_model}/h36m_3d_25frames_ckpt_epoch_{epoch+1}_q1.pt')
        # we know that in this way we save only the parameters, but we tried using torch.jit.scripts and it didn't work
        # in particular, it gives us this error "RuntimeError: Can't redefine method: forward on class: __torch__.torch.nn.modules.dropout.Dropout (of Python compilation unit at: 0x57b5c8a24d70)"
        # we search it on google but we didn't find any solution on any forum, so we stuck to the standard torch.save()

        # Plot the training and validation loss

        fig, ax = plt.subplots()
        x_tick_freq = 1 if epoch < 16 else 2 if epoch<26 else 4
        epochs = range(1, epoch+2, x_tick_freq) # The epoch numbering in the plots starts from 1 in order to stick with the code above
        x_list = list(range(1, len(train_loss)+1))
        ax.plot(x_list, train_loss, 'r', label='Train loss') # Line that displays the train loss
        ax.plot(x_list, val_loss, 'g', label='Val loss') # Line that displays the validation loss
        ax.set_xticks(list(epochs))
        ax.set_xlabel('Epochs \n ' + str(epoch))
        ax.set_ylabel('Loss')
        plt.legend()
        plt.title('Loss History', fontsize=12)

        plt.savefig(path_to_save_model + "plots/loss_epoch_"+str(epoch+1)+"-"+str(lr)+"-"+str(weight_decay)+".png", bbox_inches='tight')
        plt.show()

  torch.save(best_model.state_dict(), f'{path_to_save_model}/h36m_3d_25frames_ckpt_best_custom_model.pt')

In [ ]:
save_and_plot = True # save the model and plot the loss. Change to True if you want to save the model and plot the loss

# launch training
train_custom_model(data_loader,vald_loader, path_to_save_model=checkpoint_our_model_path)
gt_preds = [] # Array to store tuples (ground-truth, prediction)

In [ ]:
def test_custom_model(ckpt_path=None):
    model = Second_Model(num_joints=joints_to_consider, n_layers=6,
                 num_frames=10, num_frames_out=10, num_heads=num_heads,
                 in_channels=3, hidden_channels=16, kernel_size=[3,3], batch_size=batch_size ,use_pes=True).to(device)
    model.load_state_dict(torch.load(ckpt_path))
    print('model loaded')
    model.eval()
    gt_preds = []
    accum_loss=0
    n_batches=0 # number of batches for all the sequences
    actions=define_actions(actions_to_consider_test)
    dim_used = np.array([6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 21, 22, 23, 24, 25,
                      26, 27, 28, 29, 30, 31, 32, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45,
                      46, 47, 51, 52, 53, 54, 55, 56, 57, 58, 59, 63, 64, 65, 66, 67, 68,
                      75, 76, 77, 78, 79, 80, 81, 82, 83, 87, 88, 89, 90, 91, 92])
    # joints at same loc
    joint_to_ignore = np.array([16, 20, 23, 24, 28, 31])
    index_to_ignore = np.concatenate((joint_to_ignore * 3, joint_to_ignore * 3 + 1, joint_to_ignore * 3 + 2))
    joint_equal = np.array([13, 19, 22, 13, 27, 30])
    index_to_equal = np.concatenate((joint_equal * 3, joint_equal * 3 + 1, joint_equal * 3 + 2))
    totalll=0
    counter=0
    for action in actions:
      running_loss=0
      n=0
      dataset_test = datasets.Datasets(data_path,input_n,output_n,skip_rate, split=2,actions=[action])

      test_loader = DataLoader(dataset_test, batch_size=256, shuffle=False, num_workers=0, pin_memory=True)
      for cnt,batch in enumerate(test_loader):
        with torch.no_grad():

          batch=batch.to(device)
          batch_dim=batch.shape[0]
          n+=batch_dim
          all_joints_seq=batch.clone()[:, input_n:input_n+output_n,:]
          #print(all_joints_seq.shape)
          if (batch_dim != batch_size):
            continue
          n+=batch_dim

          sequences_train=batch[:, 0:input_n, dim_used]#.view(-1,input_n,len(dim_used)//3,3).permute(0,3,1,2)
          sequences_gt=batch[:, input_n:input_n+output_n, :]#.view(-1,output_n,len(dim_used)//3,3)

          #print(sequences_gt.shape)
          #print(sequences_train.shape)
          empty_seq = torch.zeros([sequences_train.shape[0], sequences_train.shape[1]-1, sequences_train.shape[2]]).to(device)
          start_of_seq = torch.Tensor([1]).repeat(sequences_train.shape[0], 1, sequences_train.shape[2]).to(device)
          #print(start_of_seq.shape)
          dec_input = torch.cat((start_of_seq, empty_seq), dim=1)
          # Final decoder input is the concatenation of them along temporal dimension
          running_time = time.time()
          sequences_predict1=model(sequences_train, dec_input, True)
          sequences_predict2=model(sequences_predict1, dec_input, True)
          sequences_predict3=model(sequences_predict2[:, -6:, :], dec_input[:, :6, :], True)

          sequences_predict=torch.cat((sequences_predict1,sequences_predict2,sequences_predict3[:, :-1, :]),1).view(-1, output_n, joints_to_consider, 3)

          totalll += time.time()-running_time
          counter += 1

          all_joints_seq[:,:,dim_used] = sequences_predict.contiguous().view(-1,output_n,len(dim_used))

          all_joints_seq[:,:,index_to_ignore] = all_joints_seq[:,:,index_to_equal]


          #The code belowe has been added for the visualization task

          # Since the exercise ask to visualize 22 joints (connections found in the file at "./models/skeleton_connection.py.")
          # and the preditions already consider them, we exploit "dim_used," "index_to_ignore," and "index_to_equal" to
          # equalize the "joint to ignore" ones in the ground-truth. This allows us to plot the ground-truth and the prediction
          # together and analyze the differences.

          gt_eq_joints = sequences_gt.clone().cpu()
          gt_eq_joints[:,:,index_to_ignore] = gt_eq_joints[:,:,index_to_equal] # Equalize some nodes of the gt to plot only 22 joints out of 32
          #gt_eq_joints = gt_eq_joints[:,:,dim_used]

          gt_pred = (gt_eq_joints.view(-1, output_n, 32, 3), all_joints_seq.view(-1, output_n, 32, 3))
          gt_preds.append(tuple(gt_pred)) #Add tuples (ground-truth, prediction) to the list gt_preds

          loss=mpjpe_error(all_joints_seq.view(-1,output_n,32,3),sequences_gt.view(-1,output_n,32,3))
          #print(loss.item())
          running_loss+=loss.item()*batch_dim
          accum_loss+=loss.item()*batch_dim

      #print('loss at test subject for action : '+str(action)+ ' is: '+ str(running_loss/n))
      print(str(action),': ', str(np.round((running_loss/n),1)))
      n_batches+=n
    print('Average: '+str(np.round((accum_loss/n_batches),1)))
    print('Prediction time: ', totalll/counter)

In [ ]:
ckpt_path = './checkpoints/h36m_3d_25frames_ckpt_best_custom_model.pt' # load the best model
test_custom_model(ckpt_path)

### Results comparison

< comment :) >

|                  | Baseline model       | Iterative Model     | Our model           |
|------------------|----------------------|---------------------|---------------------|
| Walking          | 59.9                 | 74.7                | 45.9                |
| Eating           | 58.3                 | 61.8                | 37.5                |
| Smoking          | 60.2                 | 88.7                | 38.7                |
| Discussion       | 86.4                 | 78.9                | 48.0                |
| Greeting         | 77,6                 | 104.0               | 53.4                |
| Phoning          | 100.7                | 76.9                | 44.5                |
| Posing           | 113.8                | 114.3               | 65.3                |
| Purchases        | 102.6                | 101.2               | 53.3                |
| Sitting          | 88.1                 | 87.4                | 49.6                |
| Sitting down     | 109.6                | 114.7               | 67.9                |
| Taking photo     | 85.1                 | 88.4                | 50.5                |
| Waiting          | 81.4                 | 82.3                | 49.4                |
| Walking dog      | 110.6                | 111.3               | 61.2                |
| Walking together | 58.6                 | 67.5                | 42.1                |
| AVG              | 84.5                 | 87.6                | 50.0                |
| Prediction time  | 0.016339437663555147 | 0.04167355961269802 | 0.29130565325419105 |